In [ ]:
import numpy as np
import matplotlib.pyplot as plt
try:
    import cupy as cp
    print("CuPy available - Using GPU acceleration")
    xp = cp
except ImportError:
    print("CuPy not available - Using CPU (NumPy)")
    xp = np

In [ ]:
class NeuralNetwork:
    def __init__(self, layer_sizes, use_gpu=False, activation_method='relu'):
        self.weights = []
        self.biases = []
        self.layer_sizes = layer_sizes
        self.xp = xp if use_gpu else np
        self.activation_method = activation_method

        for i in range((len(self.layer_sizes)) - 1):
            if activation_method == 'relu':
                scale = self.xp.sqrt(2.0 / self.layer_sizes[i])
            elif activation_method == 'sigmoid':
                scale = self.xp.sqrt(1.0 / self.layer_sizes[i])
            w = self.xp.random.randn(self.layer_sizes[i], layer_sizes[i + 1]) * scale
            self.weights.append(w)
            b = self.xp.zeros((1, self.layer_sizes[i + 1]))
            self.biases.append(b)

            print(f"    Created weights from layer {i} to {i + 1} with shape: {w.shape}")
            print(f"    Created biases for layer {i + 1} with shape: {b.shape}")
    
    def sigmoid(self, x):
        return 1 / (1 + self.xp.exp(-self.xp.clip(x, -500, 500)))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def relu(self, x):
        return self.xp.maximum(0, x)
    
    def relu_derivative(self, X):
        return (X > 0).astype(float)
    
    def forward(self, X):
        """Forward propagation""" 
        activation = X
        for i in range(len(self.weights)):
            z = activation @ self.weights[i] + self.biases[i]
            activation = self.sigmoid(z)
        return activation
    
    def forward_with_activation(self, X, activation_method='relu'):
        """forward pass with relu for hidden layers, sigmoid for output"""
        activation = X
        activations = [X]

        #Hidden layers: using relu
        for i in range(len(self.weights) - 1):
            z = activation @ self.weights[i] + self.biases[i]
            
            if activation_method == 'relu':
                activation = self.relu(z)
            elif activation_method == 'sigmoid':
                activation = self.sigmoid(z)
            else:
                raise ValueError(f"Unknown activation method {activation_method}")
            activations.append(activation)
        
        #Output layer always uses sigmoid for classification
        z = activation @ self.weights[-1] + self.biases[-1]
        activation = self.sigmoid(z)
        activations.append(activation)

        return activation, activations

    def backward_with_activation(self, activations, delta, lr=0.1, activation_method='relu'):
        m = activations[0].shape[0]
        
        for i in range(len(self.weights) - 1, -1, -1):
            dw = activations[i].T @ delta / m
            db = self.xp.sum(delta, axis=0, keepdims=True) / m

            #Update parameters
            self.weights[i] = self.weights[i] - lr * dw
            self.biases[i] = self.biases[i] - lr * db

            if i > 0:
                if activation_method == 'relu':
                    delta = (delta @ self.weights[i].T) * self.relu_derivative(activations[i])
                elif activation_method == 'sigmoid':
                    delta = (delta @ self.weights[i].T) * self.sigmoid_derivative(activations[i])
    
    def predict(self, X, activation_method='relu'):
        """Make prediction"""
        if self.xp != np:
            X = self.xp.asarray(X)
        output, _ = self.forward_with_activation(X, activation_method=activation_method)
        if self.xp != np:
            output = self.xp.asnumpy(output)
        return output
    
    def train(self, X, y, epochs=100, lr=0.1, batch_size=None, activation_method='relu', verbose=True):
        if self.xp != np:
            X = self.xp.asarray(X)
            y = self.xp.asarray(y)

        losses = []
        n_samples = X.shape[0]

        use_minibatch = batch_size is not None and batch_size < n_samples
        n_batches = n_samples // batch_size if use_minibatch else 1

        for epoch in range(epochs):
            epoch_loss = 0

            if use_minibatch:
                #Shuffle data
                indices = self.xp.random.permutation(n_samples)
                X_shuffled = X[indices]
                y_shuffled = y[indices]
            else:
                X_shuffled = X
                y_shuffled = y

            for i in range(n_batches):
                if use_minibatch: 
                    start = i * batch_size
                    end = start + batch_size
                    X_batch = X_shuffled[start:end]
                    y_batch = y_shuffled[start:end]
                else:
                    X_batch = X_shuffled
                    y_batch = y_shuffled

                #Forward pass
                output, activations = self.forward_with_activation(X_batch, activation_method=activation_method)

                #Compute loss
                batch_loss = float(self.xp.mean((output - y_batch) ** 2))
                epoch_loss += batch_loss

                #Backward loss
                delta = output - y_batch
                self.backward_with_activation(activations, delta, lr, activation_method=activation_method)

            avg_loss = epoch_loss / n_batches
            losses.append(avg_loss)
            
            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch}/{epochs}, Loss: {avg_loss:.6f}")
    
        return losses


nn = NeuralNetwork(layer_sizes=[2, 3, 1])
print("\n--- Inspecting Weights ---")
for i, w in enumerate(nn.weights):
    print(f"Weights for connection {i}: \n{w}")

print("\n--- Inspecting Biases ---")
for i, b in enumerate(nn.biases):
    print(f"Biases for layer {i + 1}: \n{b}")
        

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target
len(y)

In [ ]:
y_onehot = np.zeros((len(y), 3))
for i in range(len(y)):
    y_onehot[i, y[i]] = 1
print(f"X shape: {X.shape}")  # (150, 4)
print(f"y shape: {y_onehot.shape}")  # (150, 3)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
print(f"=== Creating Neural Network ===")
nn = NeuralNetwork(layer_sizes=[4, 8, 3])

print(f"=== Training ===")
losses = nn.train(X_train, y_train, epochs=2000, lr=0.1, batch_size=32, verbose=True)

In [ ]:
y_pred = nn.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_labels == y_test_labels) * 100
print(f"=== Results ===")
print(f"Test accuracy: {accuracy:.3f}%")

for i in range(5):
    print(f"True: {y_test_labels[i]}, Predict: {y_pred_labels[i]}, Probability: {y_pred[i]}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss - Iris Dataset")
plt.grid(True)
plt.show()

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test_labels, y_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, 
            yticklabels=iris.target_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X_mnist = mnist.data.to_numpy()
y_mnist = mnist.target.to_numpy().astype(int)

print(f"X shape: {X_mnist.shape}")
print(f"y shape: {y_mnist.shape}")

In [ ]:
#Normalize pixel values
X_mnist = X_mnist / 255.0

#Onehot encoding
y_onehot = np.zeros((len(y_mnist), 10))
for i in range(len(y_mnist)):
    y_onehot[i, y_mnist[i]] = 1

print(f"X range: [{X_mnist.min()}, {X_mnist.max()}]")
print(f"y_onehot shape: {y_onehot.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_mnist, y_onehot, test_size=0.2, random_state=42)
print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
print("\n=== Creating Neural Network ===")
nn_mnist = NeuralNetwork(layer_sizes=[784,256, 128, 64, 10], use_gpu=True)
print("\n=== Training ===")
losses = nn_mnist.train(X_train, y_train, epochs=100, lr=0.001, verbose=True)

In [ ]:
y_pred = nn_mnist.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_labels == y_test_labels) * 100
print(f"Accuracy: {accuracy:.4f}%")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss - MNIST Dataset')  
plt.grid(True)
plt.show()

In [ ]:
print("\n=== Creating Neural Network ===")
nn_mnist_minibatch = NeuralNetwork(layer_sizes=[784, 256, 128, 64, 10], use_gpu=True, activation_method='relu')
print("\n=== Training ===")
losses = nn_mnist_minibatch.train(X_train, y_train, epochs=150, batch_size=128, lr=0.01, activation_method='relu', verbose=True)

# MNIST Training Results Summary

## Experiment Results

### Experiment 1: Baseline Network
- **Architecture:** `[784, 128, 64, 10]`
- **Activation:** ReLU
- **Training:**
  - Epochs: 100
  - Batch Size: 128
  - Learning Rate: 0.005
- **Accuracy:** `96.7857%` 

---

### Experiment 2: Higher Learning Rate
- **Architecture:** `[784, 128, 64, 10]`
- **Activation:** ReLU
- **Training:**
  - Epochs: 150
  - Batch Size: 128
  - Learning Rate: 0.01 
- **Accuracy:** `97.5714%`  (+0.79%)

---

### Experiment 3: Deeper Network
- **Architecture:** `[784, 256, 128, 64, 10]` 
- **Activation:** ReLU
- **Training:**
  - Epochs: 150
  - Batch Size: 128
  - Learning Rate: 0.01
- **Accuracy:** `97.6929%`  (+0.12%)

In [ ]:
y_pred = nn_mnist_minibatch.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_labels == y_test_labels) * 100
print(f"Accuracy: {accuracy:.4f}%")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss - MNIST Dataset (Minibatch)')
plt.grid(True)
plt.show()

In [ ]:

cm = confusion_matrix(y_test_labels, y_pred_labels)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('ed Label')
plt.ylabel('True Label')
plt.title(f'Confusion Matrix - Accuracy: {accuracy:.2f}%')
plt.show()

In [ ]:
print("\n=== Per-Digit Accuracy ===")
for digit in range(10):
    mask = y_test_labels == digit
    digit_acc = np.mean(y_pred_labels[mask] == digit) * 100
    print(f"Digit {digit}: {digit_acc:.2f}%")